<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/t81_558_class_02_3_pandas_grouping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks

**Module 2: Python for Machine Learning**

- Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
- For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).


# Module 2 Material

Main video lecture:

- Part 2.1: Introduction to Pandas [[Video]](https://www.youtube.com/watch?v=wixHCvnvnsU&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_02_1_python_pandas.ipynb)
- Part 2.2: Categorical Values [[Video]](https://www.youtube.com/watch?v=Fm7Ax23hDP0&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_02_2_pandas_cat.ipynb)
- **Part 2.3: Grouping, Sorting, and Shuffling in Python Pandas** [[Video]](https://www.youtube.com/watch?v=tUhaD8xWd7k&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_02_3_pandas_grouping.ipynb)
- Part 2.4: Using Apply and Map in Pandas [[Video]](https://www.youtube.com/watch?v=YNo_mg1RrkM&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_02_4_pandas_functional.ipynb)
- Part 2.5: Feature Engineering in Pandas for Deep Learning in PyTorch [[Video]](https://www.youtube.com/watch?v=ezaVtM405Qs&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_02_5_pandas_features.ipynb)


# Part 2.3: Grouping, Sorting, and Shuffling

We will take a look at a few ways to affect an entire Pandas data frame. These techniques will allow us to group, sort, and shuffle data sets. These are all essential operations for both data preprocessing and evaluation.

## Shuffling a Dataset

There may be information lurking in the order of the rows of your dataset. Unless you are dealing with time-series data, the order of the rows should not be significant. Consider if your training set included employees in a company. Perhaps this dataset is ordered by the number of years the employees were with the company. It is okay to have an individual column that specifies years of service. However, having the data in this order might be problematic.

Consider if you were to split the data into training and validation. You could end up with your validation set having only the newer employees and the training set longer-term employees. Separating the data into a k-fold cross validation could have similar problems. Because of these issues, it is important to shuffle the data set.

Often shuffling and reindexing are both performed together. Shuffling randomizes the order of the data set. However, it does not change the Pandas row numbers. The following code demonstrates a reshuffle. Notice that the program has not reset the row indexes' first column. Generally, this will not cause any issues and allows tracing back to the original order of the data. However, I usually prefer to reset this index. I reason that I typically do not care about the initial position, and there are a few instances where this unordered index can cause issues.


In [2]:
import numpy as np
import pandas as pd

In [9]:
url = "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv"

df = pd.read_csv(
    url, na_values=["NA", "?"]
)

In [4]:
# Create a new random generator
rng = np.random.default_rng(43)  # Use 43 for reproducibility, or leave it out for a different shuffle each time

In [7]:
pd.set_option("display.max_columns", 7)
pd.set_option("display.max_rows", 5)

def tweak_data(data):
    return (data
            .pipe(lambda x: x.reindex(rng.permutation(x.index)))
           )

df = tweak_data(df)
df

,mpg,cylinders,displacement,...,year,origin,name
157,15.0,8,350.0,...,75,1,chevrolet bel air
162,15.0,6,258.0,...,75,1,amc matador
...,...,...,...,...,...,...,...
108,20.0,4,97.0,...,73,3,toyota carina
121,15.0,8,318.0,...,73,1,dodge dart custom


The following code demonstrates a reindex. Notice how the reindex orders the row indexes.


In [10]:
def tweak_data(data):
    return (data
            .pipe(lambda x: x.reindex(rng.permutation(x.index)))
            .reset_index(drop=True)
           )

df = tweak_data(df)
df

,mpg,cylinders,displacement,...,year,origin,name
0,28.0,4,98.0,...,72,1,dodge colt (sw)
1,19.4,8,318.0,...,78,1,dodge diplomat
...,...,...,...,...,...,...,...
396,27.0,4,140.0,...,82,1,ford mustang gl
397,28.8,6,173.0,...,79,1,chevrolet citation


### Sorting a Data Set

While it is always good to shuffle a data set before training, during training and preprocessing, you may also wish to sort the data set. Sorting the data set allows you to order the rows in either ascending or descending order for one or more columns. The following code sorts the MPG dataset by name and displays the first car.


In [11]:
df = pd.read_csv(
    url, na_values=["NA", "?"]
)

In [12]:
def tweak_data(data):
    return (data
            .sort_values(by="name", ascending=True)
            .reset_index(drop=True)
           )

df = tweak_data(df)
print(f"The first car is: {df['name'].iloc[0]}")
df

The first car is: amc ambassador brougham


,mpg,cylinders,displacement,...,year,origin,name
0,13.0,8,360.0,...,73,1,amc ambassador brougham
1,15.0,8,390.0,...,70,1,amc ambassador dpl
...,...,...,...,...,...,...,...
396,44.3,4,90.0,...,80,2,vw rabbit c (diesel)
397,31.9,4,89.0,...,79,2,vw rabbit custom


### Grouping a Data Set

Grouping is a typical operation on data sets. Structured Query Language (SQL) calls this operation a "GROUP BY." Programmers use grouping to summarize data. Because of this, the summarization row count will usually shrink, and you cannot undo the grouping. Because of this loss of information, it is essential to keep your original data before the grouping.

We use the Auto MPG dataset to demonstrate grouping.


In [13]:
df = pd.read_csv(
    url, na_values=["NA", "?"]
)

df

,mpg,cylinders,displacement,...,year,origin,name
0,18.0,8,307.0,...,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,...,70,1,buick skylark 320
...,...,...,...,...,...,...,...
396,28.0,4,120.0,...,82,1,ford ranger
397,31.0,4,119.0,...,82,1,chevy s-10


You can use the above data set with the group to perform summaries. For example, the following code will group cylinders by the average (mean). This code will provide the grouping. In addition to **mean**, you can use other aggregating functions, such as **sum** or **count**.


In [16]:
def data_groupby(data, col_1="cylinders", col_2="mpg", agg_method='mean'):
    # Validate if the provided aggregation method is callable on a GroupBy object
    if not hasattr(pd.core.groupby.SeriesGroupBy, agg_method):
        raise ValueError(f"Invalid aggregation method: {agg_method}")

    return (data
            .groupby(col_1)[col_2]
            .agg(agg_method)  # Use the agg() method with the specified aggregation function
           )

g = data_groupby(df)
g

cylinders
3    20.550000
4    29.286765
5    27.366667
6    19.985714
8    14.963107
Name: mpg, dtype: float64

It might be useful to have these **mean** values as a dictionary.


In [17]:
def data_groupby(data, col_1="cylinders", col_2="mpg", agg_method='mean'):
    # Validate if the provided aggregation method is callable on a GroupBy object
    if not hasattr(pd.core.groupby.SeriesGroupBy, agg_method):
        raise ValueError(f"Invalid aggregation method: {agg_method}")

    return (data
            .groupby(col_1)[col_2]
            .agg(agg_method)  # Use the agg() method with the specified aggregation function
            .to_dict()
           )

d = data_groupby(df)
d

{3: 20.55,
 4: 29.28676470588235,
 5: 27.366666666666664,
 6: 19.985714285714284,
 8: 14.963106796116506}

A dictionary allows you to access an individual element quickly. For example, you could quickly look up the mean for six-cylinder cars. You will see that target encoding, introduced later in this module, uses this technique.


In [19]:
d[8]

14.963106796116506

The code below shows how to count the number of rows that match each cylinder count.


In [22]:
def data_groupby(data, col_1="cylinders", col_2="mpg", agg_method='mean'):
    # Validate if the provided aggregation method is callable on a GroupBy object
    if not hasattr(pd.core.groupby.SeriesGroupBy, agg_method):
        raise ValueError(f"Invalid aggregation method: {agg_method}")

    return (data
            .groupby(col_1)[col_2]
            .agg(agg_method)  # Use the agg() method with the specified aggregation function
            .to_dict()
           )

c = data_groupby(df, agg_method='count')
c

{3: 4, 4: 204, 5: 3, 6: 84, 8: 103}